# Sunbird RC API - Water Facility Demo

This notebook demonstrates how to interact with Sunbird RC Registry API for Water Facilities.

## Prerequisites
- Sunbird RC services running (use `./start-sunbird.sh`)
- Registry API available at http://localhost:8081
- WaterFacility schema configured

## Setup

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
BASE_URL = "http://localhost:8081/api/v1"
HEADERS = {"Content-Type": "application/json"}

print("✅ Setup complete!")
print(f"Registry API: {BASE_URL}")

✅ Setup complete!
Registry API: http://localhost:8081/api/v1


## 1. Check Registry Health

In [2]:
response = requests.get("http://localhost:8081/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

Status: True
Service: sunbirdrc-registry-api


<IPython.core.display.JSON object>

## 2. List All Water Facilities

In [3]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=HEADERS)
data = response.json()

print(f"Total Water Facilities: {data['totalCount']}\n")

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    # Select key columns (only include columns that exist)
    desired_columns = ['wfName', 'wfId', 'typeOfWaterFacility', 'institution', 'foundingDate', 'waterCapacity', 'osid']
    columns = [col for col in desired_columns if col in df.columns]
    display(df[columns])
    
    # Show location details separately
    if 'location' in df.columns:
        print("\nLocation Details:")
        location_df = pd.json_normalize(df['location'])
        location_df['wfName'] = df['wfName'].values
        display(location_df[['wfName', 'province', 'district', 'village']])
else:
    print("No water facilities found")

Total Water Facilities: 0

No water facilities found


## 3. Create a New Water Facility

In [4]:
new_facility = {
    "wfName": "IPAM Dago Pakar",
    "typeOfWaterFacility": "Water Treatment Plant",
    "location": {
        "province": "Jawa Barat",
        "district": "Kota Bandung",
        "village": "Dago"
    },
    "address": "Jl. Dago Pakar No. 12, Bandung 40135",
    "institution": "PDAM Tirtawening Kota Bandung",
    "foundingDate": "2018-06-15",
    "waterCapacity": 50000
}

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=HEADERS,
    json=new_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print("✅ Water Facility created successfully!")
    print(f"osid: {osid}")
    
    # Fetch the created facility to show the generated wfId
    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
    if fetch_response.status_code == 200:
        created_facility = fetch_response.json()
        print(f"wfId: {created_facility.get('wfId', 'N/A')}")
        print(f"\nGenerated wfId format: WF-<PROVINCE>-<DISTRICT>-<TYPE>-<HASH>")
        display(JSON(created_facility, expanded=True))
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

✅ Water Facility created successfully!
osid: 1-5f8279aa-a5de-457f-a282-c02a309805bb
wfId: WF-JAW-KOT-WTP-66EB8D

Generated wfId format: WF-<PROVINCE>-<DISTRICT>-<TYPE>-<HASH>


<IPython.core.display.JSON object>

## 4. Get a Specific Water Facility by ID

In [5]:
# Get the first facility's ID
response = requests.get(f"{BASE_URL}/WaterFacility")
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    print(f"Fetching water facility: {facility_id}\n")
    response = requests.get(f"{BASE_URL}/WaterFacility/{facility_id}")
    facility = response.json()
    display(JSON(facility, expanded=True))
else:
    print("No facilities found")

Fetching water facility: 1-5f8279aa-a5de-457f-a282-c02a309805bb



<IPython.core.display.JSON object>

## 5. Update a Water Facility

In [6]:
# Get first facility
response = requests.get(f"{BASE_URL}/WaterFacility")
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    
    # Update the facility
    update_data = facilities[0].copy()
    update_data['waterCapacity'] = 75000  # Update capacity
    update_data['address'] = '456 New River Road, Metro City, MC 12345'  # Update address
    
    response = requests.put(
        f"{BASE_URL}/WaterFacility/{facility_id}",
        headers=HEADERS,
        json=update_data
    )
    
    if response.status_code == 200:
        print("✅ Water Facility updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
else:
    print("No facilities available to update")

✅ Water Facility updated successfully!


<IPython.core.display.JSON object>

## 6. Search Water Facilities

In [7]:
search_query = {
    "filters": {
        "typeOfWaterFacility": {"eq": "Water Treatment Plant"}
    }
}

response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Water Treatment Plants\n")

    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        # Include wfId in the display
        desired_columns = ['wfId', 'wfName', 'typeOfWaterFacility', 'institution', 'waterCapacity']
        columns = [col for col in desired_columns if col in df.columns]
        display(df[columns])

        # Show location details
        if 'location' in df.columns:
            print("\nLocation Details:")
            location_df = pd.json_normalize(df['location'])
            location_df['wfName'] = df['wfName'].values
            if 'wfId' in df.columns:
                location_df['wfId'] = df['wfId'].values
                display(location_df[['wfId', 'wfName', 'province', 'district', 'village']])
            else:
                display(location_df[['wfName', 'province', 'district', 'village']])
    else:
        print("No results found")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

Found 1 Water Treatment Plants



,wfId,wfName,typeOfWaterFacility,institution,waterCapacity
0,WF-JAW-KOT-WTP-66EB8D,IPAM Dago Pakar,Water Treatment Plant,PDAM Tirtawening Kota Bandung,75000



Location Details:


,wfId,wfName,province,district,village
0,WF-JAW-KOT-WTP-66EB8D,IPAM Dago Pakar,Jawa Barat,Kota Bandung,Dago


## 7. Bulk Create Water Facilities

In [8]:
# Sample bulk data with Indonesian locations
bulk_facilities = [
    {
        "wfName": "Waduk Jatiluhur",
        "typeOfWaterFacility": "Reservoir",
        "location": {
            "province": "Jawa Barat",
            "district": "Kabupaten Purwakarta",
            "village": "Jatiluhur"
        },
        "address": "Jl. Waduk Jatiluhur, Purwakarta 41152",
        "institution": "Perum Jasa Tirta II",
        "foundingDate": "1967-08-26",
        "waterCapacity": 1000000
    },
    {
        "wfName": "Pompa Distribusi Cikokol",
        "typeOfWaterFacility": "Pumping Station",
        "location": {
            "province": "Banten",
            "district": "Kota Tangerang",
            "village": "Cikokol"
        },
        "address": "Jl. MH Thamrin No. 45, Tangerang 15117",
        "institution": "PDAM Tirta Benteng Kota Tangerang",
        "foundingDate": "2015-03-10",
        "waterCapacity": 25000
    },
    {
        "wfName": "IPA Buaran",
        "typeOfWaterFacility": "Water Treatment Plant",
        "location": {
            "province": "DKI Jakarta",
            "district": "Jakarta Timur",
            "village": "Klender"
        },
        "address": "Jl. Raya Bekasi KM 18, Jakarta Timur 13930",
        "institution": "PAM Jaya",
        "foundingDate": "1990-05-20",
        "waterCapacity": 450000
    },
    {
        "wfName": "SWRO Nusa Penida",
        "typeOfWaterFacility": "Desalination Plant",
        "location": {
            "province": "Bali",
            "district": "Kabupaten Klungkung",
            "village": "Ped"
        },
        "address": "Desa Ped, Nusa Penida, Klungkung 80771",
        "institution": "PDAM Tirta Tohlangkir",
        "foundingDate": "2021-11-15",
        "waterCapacity": 5000
    },
    {
        "wfName": "Reservoir Cilandak",
        "typeOfWaterFacility": "Distribution Center",
        "location": {
            "province": "DKI Jakarta",
            "district": "Jakarta Selatan",
            "village": "Cilandak Barat"
        },
        "address": "Jl. Cilandak KKO, Jakarta Selatan 12560",
        "institution": "PAM Jaya",
        "foundingDate": "2010-09-01",
        "waterCapacity": 80000
    }
]

print(f"Creating {len(bulk_facilities)} water facilities...\n")

results = []
for i, facility in enumerate(bulk_facilities, 1):
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=HEADERS,
        json=facility
    )

    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']
        
        # Fetch the created facility to get the generated wfId
        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
        wfId = "N/A"
        if fetch_response.status_code == 200:
            created = fetch_response.json()
            wfId = created.get('wfId', 'N/A')
        
        print(f"{i}. {facility['wfName']}")
        print(f"   wfId: {wfId}")
        print(f"   osid: {osid}\n")
        results.append({'status': 'success', 'osid': osid, 'wfId': wfId, 'name': facility['wfName']})
    else:
        print(f"{i}. {facility['wfName']} - FAILED ({response.status_code})")
        print(f"   Error: {response.text}\n")
        results.append({'status': 'failed', 'name': facility['wfName'], 'error': response.text})

print(f"{'='*50}")
print(f"Created {len([r for r in results if r['status'] == 'success'])} facilities")
print(f"Failed {len([r for r in results if r['status'] == 'failed'])} facilities")

Creating 5 water facilities...

1. Waduk Jatiluhur
   wfId: WF-JAW-KAB-RES-CB37DF
   osid: 1-179ff6e9-970e-4c4f-a0d6-510c6db5221e

2. Pompa Distribusi Cikokol
   wfId: WF-BAN-KOT-PS-AFD2B8
   osid: 1-6b7ab912-c32b-4b16-98e5-dc9db3808f52

3. IPA Buaran
   wfId: WF-DKI-JAK-WTP-846A62
   osid: 1-8d966e54-d442-4b8d-8942-1b8ed17dfb8c

4. SWRO Nusa Penida
   wfId: WF-BAL-KAB-DES-77CAA8
   osid: 1-16cea628-3a27-4d47-b379-804a338ff763

5. Reservoir Cilandak
   wfId: WF-DKI-JAK-DC-485314
   osid: 1-d303a022-e9a6-4c0c-ba4f-25a3c986d88b

Created 5 facilities
Failed 0 facilities


## 7.1 Test Duplicate Rejection (wfId uniqueness)

The wfId is generated based on a hash of facility attributes (wfName, typeOfWaterFacility, location).
Attempting to create a facility with the same attributes should be rejected due to the unique constraint on wfId.

In [9]:
# Test: Attempt to create a duplicate facility
# This should fail because wfId is unique based on facility attributes

duplicate_facility = {
    "wfName": "Waduk Jatiluhur",  # Same name as bulk create
    "typeOfWaterFacility": "Reservoir",  # Same type
    "location": {
        "province": "Jawa Barat",  # Same location
        "district": "Kabupaten Purwakarta",
        "village": "Jatiluhur"
    },
    "address": "Different address - should not matter",
    "institution": "Different Institution - should not matter",
    "foundingDate": "2024-01-01",  # Different date - should not matter
    "waterCapacity": 999999  # Different capacity - should not matter
}

print("Testing duplicate rejection...")
print(f"Attempting to create: {duplicate_facility['wfName']}")
print(f"Location: {duplicate_facility['location']['province']}, {duplicate_facility['location']['district']}")
print()

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=HEADERS,
    json=duplicate_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print(f"Unexpected: Facility was created with osid: {osid}")
    
    # Fetch to see the wfId
    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}")
    if fetch_response.status_code == 200:
        created = fetch_response.json()
        print(f"wfId: {created.get('wfId', 'N/A')}")
else:
    print(f"Expected behavior: Duplicate rejected!")
    print(f"Status code: {response.status_code}")
    try:
        error_detail = response.json()
        print(f"Error: {json.dumps(error_detail, indent=2)}")
    except:
        print(f"Error: {response.text}")

Testing duplicate rejection...
Attempting to create: Waduk Jatiluhur
Location: Jawa Barat, Kabupaten Purwakarta

Expected behavior: Duplicate rejected!
Status code: 500
Error: {
  "id": "sunbird-rc.registry.post",
  "ver": "1.0",
  "ets": 1767949432372,
  "params": {
    "resmsgid": "",
    "msgid": "a4c0c7d8-c176-41b8-b0f2-2e44e03593a2",
    "err": "",
    "status": "UNSUCCESSFUL",
    "errmsg": "dev.sunbirdrc.registry.exception.UniqueIdentifierException: dev.sunbirdrc.registry.exception.UniqueIdentifierException$GenerateException: Unable to generate id: Duplicate WaterFacility: A facility with wfId 'WF-JAW-KAB-RES-CB37DF' already exists. Facilities with the same name, type, and location are not allowed."
  },
  "responseCode": "OK",
  "result": {}
}


## 8. Export to CSV

In [10]:
response = requests.get(f"{BASE_URL}/WaterFacility")
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    
    # Flatten location object for export
    if 'location' in df.columns:
        location_df = pd.json_normalize(df['location'])
        location_df.columns = [f'location_{col}' for col in location_df.columns]
        df = pd.concat([df.drop('location', axis=1), location_df], axis=1)
    
    # Select columns to export (updated for new schema)
    export_columns = [
        'wfName', 'wfId', 'typeOfWaterFacility', 'institution',
        'location_province', 'location_district', 'location_village',
        'address', 'foundingDate', 'waterCapacity', 
        'osid', 'osCreatedAt'
    ]
    
    # Only include columns that exist in the dataframe
    available_columns = [col for col in export_columns if col in df.columns]
    
    filename = f'water_facilities.csv'
    df[available_columns].to_csv(filename, index=False)
    
    print(f"✅ Exported {len(df)} water facilities to: {filename}")
    display(df[available_columns].head())
else:
    print("No water facilities to export")

✅ Exported 6 water facilities to: water_facilities.csv


,wfName,wfId,typeOfWaterFacility,institution,location_province,location_district,location_village,address,foundingDate,waterCapacity,osid,osCreatedAt
0,IPAM Dago Pakar,WF-JAW-KOT-WTP-66EB8D,Water Treatment Plant,PDAM Tirtawening Kota Bandung,Jawa Barat,Kota Bandung,Dago,"456 New River Road, Metro City, MC 12345",2018-06-15,75000,1-5f8279aa-a5de-457f-a282-c02a309805bb,2026-01-09T09:03:52.156Z
1,Waduk Jatiluhur,WF-JAW-KAB-RES-CB37DF,Reservoir,Perum Jasa Tirta II,Jawa Barat,Kabupaten Purwakarta,Jatiluhur,"Jl. Waduk Jatiluhur, Purwakarta 41152",1967-08-26,1000000,1-179ff6e9-970e-4c4f-a0d6-510c6db5221e,2026-01-09T09:03:52.271Z
2,IPA Buaran,WF-DKI-JAK-WTP-846A62,Water Treatment Plant,PAM Jaya,DKI Jakarta,Jakarta Timur,Klender,"Jl. Raya Bekasi KM 18, Jakarta Timur 13930",1990-05-20,450000,1-8d966e54-d442-4b8d-8942-1b8ed17dfb8c,2026-01-09T09:03:52.310Z
3,Pompa Distribusi Cikokol,WF-BAN-KOT-PS-AFD2B8,Pumping Station,PDAM Tirta Benteng Kota Tangerang,Banten,Kota Tangerang,Cikokol,"Jl. MH Thamrin No. 45, Tangerang 15117",2015-03-10,25000,1-6b7ab912-c32b-4b16-98e5-dc9db3808f52,2026-01-09T09:03:52.293Z
4,Reservoir Cilandak,WF-DKI-JAK-DC-485314,Distribution Center,PAM Jaya,DKI Jakarta,Jakarta Selatan,Cilandak Barat,"Jl. Cilandak KKO, Jakarta Selatan 12560",2010-09-01,80000,1-d303a022-e9a6-4c0c-ba4f-25a3c986d88b,2026-01-09T09:03:52.351Z


## 9. Delete All Water Facilities

In [11]:
# Delete all Water Facilities
response = requests.get(f"{BASE_URL}/WaterFacility")
facilities = response.json()['data']

if facilities:
    print(f"Deleting {len(facilities)} water facilities...\n")
    
    deleted = 0
    failed = 0
    for facility in facilities:
        osid = facility['osid']
        name = facility['wfName']
        
        response = requests.delete(f"{BASE_URL}/WaterFacility/{osid}")
        
        if response.status_code == 200:
            print(f"✅ Deleted: {name} ({osid})")
            deleted += 1
        else:
            print(f"❌ Failed to delete: {name} ({osid})")
            failed += 1
    
    print(f"\n✅ Deleted {deleted} facilities")
    if failed > 0:
        print(f"❌ Failed {failed} facilities")
else:
    print("No water facilities to delete")

Deleting 6 water facilities...

✅ Deleted: IPAM Dago Pakar (1-5f8279aa-a5de-457f-a282-c02a309805bb)
✅ Deleted: Waduk Jatiluhur (1-179ff6e9-970e-4c4f-a0d6-510c6db5221e)
✅ Deleted: IPA Buaran (1-8d966e54-d442-4b8d-8942-1b8ed17dfb8c)
✅ Deleted: Pompa Distribusi Cikokol (1-6b7ab912-c32b-4b16-98e5-dc9db3808f52)
✅ Deleted: Reservoir Cilandak (1-d303a022-e9a6-4c0c-ba4f-25a3c986d88b)
✅ Deleted: SWRO Nusa Penida (1-16cea628-3a27-4d47-b379-804a338ff763)

✅ Deleted 6 facilities


## Summary

This notebook demonstrated:
- Checking registry health
- Listing all water facilities
- Creating new water facilities with auto-generated wfId
- Reading specific facilities
- Updating facilities
- Searching facilities by type
- Bulk operations with wfId display
- Testing duplicate rejection (wfId uniqueness)
- Exporting to CSV
- Deleting all facilities

### wfId Generation

The `wfId` is automatically generated when creating a WaterFacility using the format:
```
WF-<PROVINCE_ABBR>-<DISTRICT_ABBR>-<TYPE_CODE>-<HASH>
```

**Example:** `WF-JAW-KAB-RES-CB37DF`

**Components:**
- `WF-` - Fixed prefix for Water Facility
- `PROVINCE_ABBR` - First 3 alphanumeric characters of province (uppercase)
- `DISTRICT_ABBR` - First 3 alphanumeric characters of district (uppercase)
- `TYPE_CODE` - Facility type code:
  - `WTP` - Water Treatment Plant
  - `RES` - Reservoir
  - `PS` - Pumping Station
  - `DC` - Distribution Center
  - `DES` - Desalination Plant
- `HASH` - 6-character SHA-256 hash of normalized attributes

**Hash inputs (normalized to lowercase, trimmed):**
- wfName
- typeOfWaterFacility
- location.province
- location.district
- location.village

**Uniqueness:** The wfId is marked as a unique index field. Duplicate facilities (same core attributes) will be rejected.

### Water Facility Schema Fields
**Required Fields:**
- `wfName` - Name of the facility
- `typeOfWaterFacility` - Type (enum)
- `location` - Administrative location object
  - `province` - Province name
  - `district` - District/Regency name
  - `village` - Village/Sub-district name
- `institution` - Responsible institution
- `foundingDate` - Date of establishment (YYYY-MM-DD)

**Auto-generated Fields:**
- `wfId` - System-generated unique ID (format: WF-XXX-XXX-TYPE-HASH)

**Optional Fields:**
- `address` - Free-text address
- `waterCapacity` - Capacity in cubic meters (0-1,000,000)

### Water Facility Types Supported
- Water Treatment Plant (WTP)
- Reservoir (RES)
- Pumping Station (PS)
- Distribution Center (DC)
- Desalination Plant (DES)

For more information, see the WaterFacility schema at `java/registry/src/main/resources/public/_schemas/WaterFacility.json`